## Introduction
In this article we will discuss a very common *decision problem*: how best to travel from a starting point `A` to some target destination `Z` in the *"best"* manner possible. Of course, as we previously discussed in the [last article](https://diogenesanalytics.com/blog/2025/02/26/reasoning-about-utility), it is the *utility* that must be defined in order to *quantitatively* define what is best. The majority of the article will present the *mathematical* definition of the utility function $U(x)$ for the *travel problem*, as well as the reasoning behind the definition, and finally a simple application.

## Utility Function Defined
Before we really get into explaining the *reasoning* behind our choice of *function definition*, we will first simply show the mathematical formula in all its glory:

$$
U(x) = \frac{D_{\text{ideal}}}{M \cdot T \cdot S \cdot D_{\text{actual}}}
$$

Where:

- $D_{\text{ideal}}$ is the ideal (straight-line) distance between the start and end points of the journey.
- $D_{\text{actual}}$ is the actual traveled distance.
- $M$ is the monetary cost.
- $T$ is the total time.
- $S$ is the stress.

In the next section we will explore the *"reasoning"* behind this equation.

## Reasoning About Travel Utility
To understand how we arrived at such an equation, we must ask ourselves a simple question: what is the purpose of travel? Simply put, *travel* is about getting from one point to another. That could be called the *"reward"* or *"benefit."* But the cost of said travel does not include just money $M$, but also time $T$, and even something more abstract, which is stress $S$.
So we can think about the *"utility"* of such a problem as simply:

$$
U(x) = \frac{\text{What You Want}}{\text{What It Costs}}
$$

In this case we want to travel some distance $D$ between two points, but we will be forced to pay some cost term that includes *money* $M$, *time* $T$, and stress $S$:

$$
U(x) = \frac{D}{M \cdot T \cdot S}
$$

## Distance Efficiency
But there is something else to consider: should utility increase or decrease as we are *"forced"* to travel **more** distance than the straight line path?

In [ ]:
# libs for downloading earth map data
import pathlib
import tempfile
import urllib.request
import zipfile
from tqdm import tqdm

# natural earth 10m cultural shape file
shape_file_url = (
    "https://naciscdn.org/naturalearth/10m/cultural/10m_cultural.zip"
)

# directory to save the downloaded file
data_dir = pathlib.Path("./data/geo/")

# directory to extract the contents of the ZIP file
extract_dir = data_dir / "natural_earth_10m_cultural"

def download_with_progress(url: str, filename: pathlib.Path) -> None:
    """Utility function to download files with progress bar."""
    # set headers
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # create request
    request = urllib.request.Request(url, headers=headers)

    # open URL
    response = urllib.request.urlopen(request)

    # get total file size
    total = int(response.getheader('Content-Length', 0))

    # set block size for updating
    block_size = 1024

    # create progress bar
    tqdm_bar = tqdm(total=total, unit='B', unit_scale=True, desc=filename.name)

    # open new file
    with open(filename, 'wb') as file:
        # loop over blocks
        while True:
            # get next data chunk
            buffer = response.read(block_size)

            # quit if finished
            if not buffer:
                break

            # write out data chunk
            file.write(buffer)

            # update progress bar
            tqdm_bar.update(len(buffer))

    # finished
    tqdm_bar.close()

# check for previous download
if not extract_dir.exists():
    # create the parent tree (in case it doesn't exist)
    data_dir.mkdir(parents=True, exist_ok=True)
    
    # create a temporary directory to download the file
    with tempfile.TemporaryDirectory() as temp_dir:
        # filename for the downloaded ZIP file
        zip_filename = pathlib.Path(temp_dir) / "10m_cultural.zip"
        
        # download the ZIP file with progress bar
        download_with_progress(shape_file_url, zip_filename)
    
        # extract the contents of the ZIP file
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)

In [ ]:
import geopandas as gpd

# get world geo data
world_map = gpd.read_file(extract_dir / "10m_cultural")

# Filter for South American countries
south_america = world_map[(world_map["CONTINENT"] == "South America")]

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString

# set initial figure counter to 1
fig_count = 1

# set the style to a dark theme
plt.style.use("dark_background")

# match website background
plt.rcParams["figure.facecolor"] = "#181818"
plt.rcParams["axes.facecolor"] = "#181818"
plt.rcParams["axes.edgecolor"] = "#181818"

# get subplot for South America
fig, ax = plt.subplots(figsize=(8, 8))

# set the view for only the mainland
ax.set_xlim([-85, -30])
ax.set_ylim([-60, 15])

# turn off axes for the South America plot only
ax.axis("off")

# plot all countries with darkish grey color
south_america.plot(
    ax=ax,
    color=plt.cm.viridis(0.67),
    edgecolor="#181818",
    linewidth=0.2,
);

# Define airport coordinates (longitude, latitude)
bogota = (-74.0721, 4.7110)          # Bogotá, Colombia (BOG)
buenos_aires = (-58.3816, -34.6037)  # Buenos Aires, Argentina (EZE)
sao_paulo = (-46.6333, -23.5505)     # São Paulo, Brazil (GRU)
lima = (-77.0428, -12.0464)          # Lima, Peru (LIM)
santiago = (-70.6483, -33.4489)      # Santiago, Chile (SCL)
santa_cruz = (-63.1805, -17.7892)    # Santa Cruz de la Sierra, Bolivia (VVI)
asuncion = (-57.5759, -25.2637)      # Asunción, Paraguay (ASU)

# Create GeoDataFrame for airports
airports = gpd.GeoDataFrame(
    {
        "City": [
            "Bogotá (BOG)", "São Paulo (GRU)", "Buenos Aires (EZE)", 
            "Lima (LIM)", "Santiago (SCL)", "Santa Cruz (VVI)",
            "Asunción (ASU)"
        ],
        "geometry": [
            Point(bogota), Point(sao_paulo), Point(buenos_aires),
            Point(lima), Point(santiago), Point(santa_cruz),
            Point(asuncion)
        ]
    },
    crs="EPSG:4326",
)

# Define different flight paths (corrected)
routes = {
    "Direct Flight": [bogota, buenos_aires],
    "Via São Paulo": [bogota, sao_paulo, buenos_aires],
    "Via Lima": [bogota, lima, buenos_aires],
    "Via Santiago": [bogota, santiago, buenos_aires],
    "Via Lima & Santiago": [bogota, lima, santiago, buenos_aires],
    "Via Lima & Asunción": [bogota, lima, asuncion, buenos_aires],
    "Via Santa Cruz & Santiago": [bogota, santa_cruz, santiago, buenos_aires],
}

# Plot flight paths with a specific label for each route
for route, path in routes.items():
    # Set color to red for Direct Flight, else a different color
    if "Direct" in route:
        color = "red"
        style = "solid"

    else:
        color = plt.cm.inferno(1.0)
        style = "dashed"
    
    # Create a LineString for the route and plot it with the specified color
    line = LineString(path)
    ax.plot([point[0] for point in line.coords], [point[1] for point in line.coords], 
            color=color, linewidth=2, linestyle=style, alpha=0.7)

# Plot airports
airports.plot(ax=ax, color="red", markersize=50, edgecolor="black")

# Add labels
for x, y, label in zip(airports.geometry.x, airports.geometry.y, airports["City"]):
    ax.text(x, y, label, fontsize=8, ha="right", color="black", bbox=dict(facecolor="white", alpha=0.7))

# Manually add the legend for "Direct Flight" and "Non-Direct Flight"
from matplotlib.lines import Line2D

legend_elements = [
    Line2D([0], [0], color="red", lw=2, linestyle="solid", label="Direct Flight"),
    Line2D([0], [0], color=plt.cm.inferno(1.0), lw=2, linestyle="dashed", label="Non-Direct Flight")
]

# Add legend
ax.legend(handles=legend_elements, loc="upper right", fontsize=8, frameon=False)

# set title
plt.suptitle(
    f"Figure {fig_count}. Bogota to Buenos Aires Flight Paths.", y=0.0001, fontsize=10,
)

# increment fig count
fig_count += 1

# adjust padding
plt.tight_layout(pad=0.5)  # Adjust the padding as needed

# display
plt.show()

In *figure 1* above we can see the various flight paths possible from *Bogota, Columbia* to *Buenos Aires, Argentina*. Clearly, any flight path that is not a *direct flight* will have a longer total distance. Should our *utility* function reward or penalize this? It would not make sense to *"reward"* excess distance, because that would be equivalent to *"waste"* distance. This can be understood in terms of **accuracy** or **efficiency**: we do not want to simply travel from point `A` to `Z` (e.g. *Bogota* to *Buenos Aires*) by any path, we want to get there by the path of *minimal waste*. So then how do we account for this *"waste distance"* because clearly we must consider it, otherwise our *utility function* would be *increasing* as the distance increases **beyond** the *"ideal distance."* The solution comes from understanding that the *"benefit"* gained from traveling is not an absolute distance but a *distance efficiency* ($E_D$):

$$
E_D = \frac{D_{\text{ideal}}}{D_{\text{actual}}}
$$

We are not *"buying"* raw distance, but instead an *"efficiency"* or *"accuracy"* of distance from our origin `A` to our target destination `Z`. Now we have the correct behavior:

- **$D_{\text{actual}} = D_{\text{ideal}}; E_D = 1$**: utility is maximized, since the denominator is just
  $M \cdot T \cdot S$, and there is no penalty from the actual distance.
- **$D_{\text{actual}} > D_{\text{ideal}}; E_D < 0$**: utility decreases as the actual distance grows larger.

We can now simplify the *utility function* even further:

$$
U(x) = \frac{E_D}{M \cdot T \cdot S}
$$

## Mathematics of Stress
The final term of the equation that we need to discuss is *stress* $S$. The *money* $M$ and *time* $T$ terms are relatively simple:

+ $M$ represents all the financial costs spent on the trip (e.g. ticket cost, baggage fees, etc ...)
+ $T$ represents all the *temporal* costs spent on the trip (e.g. flight time, layovers, traffic, delays, etc ...)

But *stress* $S$ is a bit more abstract and complex:

+ *Stress accumulates over time*, meaning that longer trips generally result in higher stress.
+ However, stress is not **linearly** dependent on time.

We *"intuitively"* feel this: a non-stop, 12-hour flight might feel more stressful than two 6-hour flights with a comfortable layover in between. Conversely, too many layovers can introduce additional stressors such as airport transfers, security checks, and unpredictability.

So how do we go about deriving the *mathematical model* of stress? Well the *"key"* insight here is to understand that *stress* is simply an example of [continuously compounding](https://en.wikipedia.org/wiki/Compound_interest) which is just a type of [exponential growth](https://en.wikipedia.org/wiki/Exponential_growth):

$$
S = S_0 e^{kT}
$$

where:  
- $S_0$ is the **baseline stress** (initial stress level).  
- $k$ is a **stress growth rate**, which could depend on factors like discomfort, unpredictability, or psychological burden.  
- $T$ is the **total time** spent on the trip.

Let's look at an example scenario that really shows what this *mathematical model* captures about *stress* $S$. For a **12-hour** direct flight:

$$
S_{\text{long-haul}} = S_0 e^{k(12)}
$$

For a trip broken into **three 4-hour segments** with **two layovers** (each reducing stress by a factor of $\alpha$, where $0 < \alpha < 1$:

1. **First Flight (4 hours)**:

$$
S_1 = S_0 e^{k(4)}
$$

2. **First Layover (partial stress reset)**:

$$
S_2 = \alpha S_1 = \alpha S_0 e^{k(4)}
$$

3. **Second Flight (4 hours)**:

$$
S_3 = S_2 e^{k(4)} = \alpha S_0 e^{k(4)} e^{k(4)} = \alpha S_0 e^{k(8)}
$$

4. **Second Layover (another stress reset)**:

$$
S_4 = \alpha S_3 = \alpha^2 S_0 e^{k(8)}
$$

5. **Final Flight (4 hours)**:

$$
S_5 = S_4 e^{k(4)} = \alpha^2 S_0 e^{k(8)} e^{k(4)} = \alpha^2 S_0 e^{k(12)}
$$

Thus, the final stress for the segmented trip is:

$$
S_{\text{segmented}} = \alpha^2 S_0 e^{k(12)}
$$

If we compare the two *travel strategies*:

1. **Long-haul flight stress**:

$$
S_{\text{long-haul}} = S_0 e^{k(12)}
$$

2. **Segmented flight stress (with layovers reducing stress by $\alpha^2$)**:

$$
S_{\text{segmented}} = \alpha^2 S_0 e^{k(12)}
$$

Since $0 < \alpha < 1$, we have:

$$
S_{\text{segmented}} < S_{\text{long-haul}}
$$

indicating that **layovers help reduce accumulated stress**, despite the total time being the same. Certainly this depends on $\alpha < 1$, otherwise we would not see such *"benefit"* from *interrupting stress growth*. To make this more clear let's look at the same example but with actual values:

- **Baseline stress level**: $S_0 = 10$ (arbitrary unit)
- **Stress growth rate**: $k = 0.1$ per hour
- **Total travel time**: $T = 12$ hours  
- **Layover stress reduction factor**: $\alpha = 0.5$ (each layover halves the accumulated stress)

First we calculate the continuous *long-haul* flight:

$$
S_{\text{long-haul}} = 10 \cdot e^{0.1 \times 12}
$$

$$
S_{\text{long-haul}} = 10 \cdot e^{1.2}
$$

$$
S_{\text{long-haul}} \approx 10 \cdot 3.32 = 33.2
$$

Now the *multi-segment* flight:

1. **First Flight (4 hours)**:

$$
S_1 = 10 \cdot e^{0.1 \times 4} = 10 \cdot e^{0.4}
$$

$$
S_1 \approx 10 \cdot 1.49 = 14.9
$$

2. **First Layover (partial stress reset)**:

$$
S_2 = 0.5 \times 14.9 = 7.45
$$

3. **Second Flight (4 hours)**:

$$
S_3 = 7.45 \cdot e^{0.1 \times 4} = 7.45 \cdot e^{0.4}
$$

$$
S_3 \approx 7.45 \cdot 1.49 = 11.1
$$

4. **Second Layover (another stress reset)**:

$$
S_4 = 0.5 \times 11.1 = 5.55
$$

5. **Final Flight (4 hours)**:

$$
S_5 = 5.55 \cdot e^{0.1 \times 4} = 5.55 \cdot e^{0.4}
$$

$$
S_5 \approx 5.55 \cdot 1.49 = 8.3
$$

The final comparison:
- **Long-haul stress**: $S_{\text{long-haul}} \approx 33.2$
- **Segmented flight stress**: $S_{\text{segmented}} \approx 8.3$

So, despite both flights covering **12 hours**, breaking it into *3 segments with 2 layovers* **reduced stress from 33.2 to 8.3**, which is a **75% reduction in final stress**! Key insights:

- A direct flight accumulates **continuous** stress exponentially.
- Layovers **reset** stress by a factor $\alpha$, preventing excessive buildup.
- Even though layovers might be inconvenient, they can significantly reduce total travel stress.

This illustrates why **ultra-long-haul flights** might feel much more exhausting compared to breaking the journey into shorter, more manageable segments.

## Growing Stress
But one aspect of the *mathematics of stress* we have not discussed is what *influences* the *growth rate* $k$? For our purposes, let us consider some *"travel examples"* in an attempt to reason about what causes *"stress"* in travel. Consider the following two scenarios:

**Scenario 1**: Empty Train
- You board a **nearly empty** train car.  
- There is **ample seating**, personal space, and no loud distractions.  
- No one is **blocking the aisles**, slowing you down, or invading your personal space.  
- The ride is **quiet**, predictable, and uninterrupted.

**Scenario 2**: Packed Subway
- You board a **rush-hour subway**, standing shoulder to shoulder with strangers.  
- Noise levels are high: **announcements, conversations, crying babies**.  
- The **air is stuffy**, and you’re forced to hold onto a metal bar with limited mobility.  
- At each stop, **more people cram in**, pushing against you.  
- The train **suddenly stops** due to delays, adding uncertainty.

There are many different ways we can *"describe"* what is the difference between the two scenarios: *noise*, *threat*, *attack*, *interruption*, *diversion*, *chaos*, *distraction*, etc ... But ultimately one word encapsulates the difference: **stimulation**. The difference between the *two scenarios* is that **scenario 1** is less stimulating. And why? There is less *stimulation* from the environment: less people, less noise, less distractions, and less interruptions. Whereas **scenario 2** is **total chaos**.

But how might we represent this *mathematically*? We simply need to account for both the *frequency* and the *amplification* of *each stimuli*:

$$
k = k_0 + \sum_{i} \sigma_i \lambda_i
$$

where:
- $k_0$ = **intrinsic stress growth rate** (baseline stress accumulation).
- $i$ = index for different types of stimuli.
- $\sigma_i$ = **stress amplification factor** for stimulus type $i$.
- $\lambda_i$ = **stimulus frequency** (events per minute, per hour, etc.).

Now we can return to the *two scenarios* above:

**Scenario 1**: Empty Train
- **Very few passengers** → low interaction frequency.  
- **Minimal announcements**.  
- **Smooth ride, few disturbances**.  

We assign reasonable values for the stress equation:  

$$
S = S_0 e^{kT}
$$

where the growth rate $k$ is influenced by **stimulus frequency**:

$$
k = \sigma_{\text{people}} \cdot \lambda_{\text{people}} + \sigma_{\text{announcements}} \cdot \lambda_{\text{announcements}} + \sigma_{\text{movement}} \cdot \lambda_{\text{movement}}
$$

Using estimated values:  

| **Stimulus**    | **Impact Factor** $\sigma$ | **Frequency** $\lambda$ (per min) |
|-----------------|-------------------|----------------|
| People moving   | $0.2$             | $1$            |
| Announcements   | $0.3$             | $0.2$          |
| Train movement  | $0.1$             | $1$            |

$$
k_{\text{empty}} = (0.2 \times 1) + (0.3 \times 0.2) + (0.1 \times 1) = 0.2 + 0.06 + 0.1 = 0.36
$$

For a **30-minute ride**, assuming $S_0 = 1$:

$$
S_{\text{empty}} = 1 \cdot e^{0.36 \times 30} \approx e^{10.8} \approx 49,402
$$  

 **Scenario 2**: Packed Subway:
 
- **Crowded conditions** → high people density.  
- **Frequent loud announcements**.  
- **Jostling, noise, and disruptions**.  

Using estimated values:  

| **Stimulus**    | **Impact Factor** $\sigma$ | **Frequency** $\lambda$ (per min) |
|------------------|------------------|----------------|
| People moving   | $0.5$         | $10$      |
| Announcements   | $0.4$         | $2$       |
| Train movement  | $0.2$         | $5$       |

$$
k_{\text{packed}} = (0.5 \times 10) + (0.4 \times 2) + (0.2 \times 5) = 5 + 0.8 + 1 = 6.8
$$

For the same **30-minute ride**, with $S_0 = 1$:

$$
S_{\text{packed}} = 1 \cdot e^{6.8 \times 30} \approx e^{204} \approx 2.27 \times 10^{88}
$$

The stress growth rate in a **packed subway** ($k = 6.8$) is exponentially higher than in an **empty train** ($k = 0.36$), leading to an *astronomical difference* in total stress over time. This highlights how a crowded, noisy environment rapidly compounds stress, while a quiet, low-stimulus setting keeps it minimal. Ultimately, **stimulus frequency**—the rate of encounters with people, noise, and movement—emerges as the key driver of stress accumulation.